TODO: 

2. Check taxonomy class, the constructor __init__ get inputs that are unused. 
3. Verify that we can load and create another notebook for Viz. 
4. Create another classifier for "hybrid" taxonomy
5. Cleanup commented functios in TissueGraph.py (they are mostly related to hybrid taxonomy) 
6. Cleanup dependencies and unused methods from Utils
7. Viz needs to support Geoms as list
8. update Viz code (mostly comments)
9. Finish configuring Sphinx
10. Push to github




# Create and construct a TMG object from multiple slices

This example notebooks performs basic TMG analysis operations. The "input" are the per slice files generated by Processing module. 

In this example, the Taxonomy/Classifiers are not given and are created on the fly based on optimal leiden. 

Overall, the steps in the analysis are: 
1. Load and create an empty TMG object
2. Create a cell layer. This loads, normalizes, creates graphs, etc, but does not classify
3. Unsupervized learning using optimal leiden to create classifier and taxonomy
4. Create iso-zone layer
5. Classify cellular neighborhoods into regions using topic modeling
6. save to file

### Import and create an empty TMG

In [1]:
from dredFISH.Analysis.TissueGraph import *
from dredFISH.Visualization.Viz import *
from dredFISH.Analysis.Classification import *

import matplotlib.pyplot as plt 

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1'
TMG = TissueMultiGraph(basepath = basepth)

### Create a cell layer

Creating a cell layer, load data from file, normalizes and creates an unclassified tissue graph

In [1]:
TMG.create_cell_layer(metric = 'cosine')
logging.info(f"TMG has {len(TMG.Layers)} Layers")

NameError: name 'TMG' is not defined

### Create Geometries

In [4]:
TMG.add_geoms()

### Create OptLeiden classifier and train it using cell layer

In [5]:
#Create the classifier
optleiden = OptimalLeidenKNNClassifier(TMG.Layers[0])

# train the classifier
optleiden.train(opt_res = 11.5,opt_params = {'iters' : 10, 'n_consensus' : 1})

# use the classifier to create types and add them to TMG using the Taxonomy created on the fly by the classifier
type_vec = optleiden.classify(TMG.Layers[0].feature_mat)


Number of types: 143 initial entropy: -6.806977295367631 number of evals: 0


In [6]:
TMG.add_type_informations(0,type_vec,optleiden.tax)

### Create isozone layer

In [7]:
TMG.create_isozone_layer()
logging.info(f"TMG has {len(TMG.Layers)} Layers")

2022-06-15 13:02:48 INFO     TMG has 2 Layers


### Create regions layer
To create regions, we first create a new classifier that works on local cell environments (local type abundace). Classify cells based on their regions and use this type to create a new layer. 

In [8]:
topic_cls = TopicClassifier(TMG.Layers[0])
topic_cls.train(max_num_of_topics = 3)
topics = topic_cls.classify(topic_cls.Env)


2022-06-15 13:03:09 INFO     Running LDA n serial


In [9]:
TMG.create_region_layer(topics,topic_cls.tax)
logging.info(f"TMG has {len(TMG.Layers)} Layers")

/home/rwollman/MyProjects/MaximallyInformativeBiocartography/dredFISH/dredFISH/Analysis/TissueGraph.py:643: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self.adata = anndata.AnnData(feature_mat) # the tissuegraph AnnData object
2022-06-15 13:06:38 INFO     TMG has 3 Layers


### Save to files
TMG is saved as a config json file, one AnnData file per layer, and one dataframe per taxonomy. 

In [10]:
TMG.save()

TypeError: save() missing 1 required positional argument: 'basepath'